In [1]:
# import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

# to avoid errors regarding browsers
header = {'User-Agent':"Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36"}

In [2]:
URLs =['https://www.chess.com/ratings/standard/all',
        'https://www.chess.com/ratings/rapid/all',
        'https://www.chess.com/ratings/blitz/all',
        'https://www.chess.com/ratings/u20',
        'https://www.chess.com/ratings/rapid/u20',
        'https://www.chess.com/ratings/blitz/u20',
        'https://www.chess.com/ratings/women',
        'https://www.chess.com/ratings/rapid/women',
        'https://www.chess.com/ratings/blitz/women'
    ]

## the process is followed for all three URLs
for URL in URLs:
    ## for naming conventtion later, we will use some part of URL
    splits = URL.split('/')
    category = splits[-1]
    format = splits[-2]

    r = requests.get(URL, headers = header)
    soup = BeautifulSoup(r.content,'html5lib')
    # print(soup.prettify())

    # get all divs from the table
    div = soup.find('div',attrs={'class':'master-players-rating-wrapper table-responsive'})
    rank = [row.text.strip() for row in div.find_all('div',attrs={'class':'master-players-rating-rank'})]
    title = [row.text for row in div.find_all('span')][3:]
    name = [row.text.strip() for row in div.find_all('a',attrs={'class':'master-players-rating-username'})]
    ratings = [row.text.strip() for row in div.find_all('div',attrs={'class':'master-players-rating-player-rank'})]
    
    ## ratings contained all three values for all formats so divide them into 
    ## three separate lists using % 
    classical = []
    rapid = []
    blitz = []

    for i in range(0, len(ratings)):
        if i%3 == 0:
            classical.append(ratings[i])
        elif i%3 == 1:
            rapid.append(ratings[i])
        else:
            blitz.append(ratings[i])
    
    ## for country there is no text present, so we will extract country values from the 
    ## attributes of the respective element
    country = []
    for row in div.find_all(attrs={'class':'master-players-rating-user-flag'}):
    
    ## beacause of political issues FIDE changed russian flag to unnamed, so we will 
    ## replace them as original for our ease
        if row['v-tooltip'] == "'Click here to see our stance on the war in Ukraine'":
            country.append('Russia')
        else:
            country.append(row['v-tooltip'])

    #Converting to DataFrame to csv file
    data = {'rank':rank,'title':title,'name':name,'country':country,'classical':classical,'rapid':rapid,'blitz':blitz}
    df = pd.DataFrame(data)
    df.to_csv(category+'_'+format+'_ratings.csv')